In [ ]:
URL= 'https://finance.yahoo.com/quote/IBM/'
headers={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"}

In [ ]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [ ]:
session=requests.session()
response=session.get(URL,headers=headers)
soup=BeautifulSoup(response.text,"html.parser")
divs = soup.findAll("div",{"class":"D(ib) Mend(20px)"})
tds = soup.findAll("td",{"data-test":"TD_VOLUME-value"})


for td in tds:
    print(td.find('span').text)


for div in divs:
    print(div.find('span').text)

In [ ]:
import pandas as pd

In [ ]:
Sentiments = pd.read_csv("MarketNewsTimeStamp.csv")



Sentiments.head()

In [ ]:
Sentiments.sort_values(by=['DateTime'],ascending = False)

In [ ]:
#Sentiments['DateTime'] = pd.to_datetime(Sentiments['Time'])

S1= Sentiments.loc[Sentiments["DateTime"] >= '2020-03-31 00:00:00']

In [ ]:
S1.head(20)

In [1]:
import pandas as pd
IBMTest = pd.read_csv('./test/IBMTest.csv')
SentimentTest =pd.read_csv('./test/sentiments.csv')
SentimentTest = SentimentTest.loc[:, ['DateTime','Polarity']].groupby(['DateTime']).mean().reset_index()
SentimentTest['DateTime'] = pd.to_datetime(SentimentTest['DateTime'])
IBMTest['DateTime'] = pd.to_datetime(IBMTest['Date'].apply(str)+' '+IBMTest['Time'])
IBMTest.drop(columns=['Date','Time'],inplace = True)
IBMTest = IBMTest.set_index('DateTime')\
          .between_time('09:00:00', '16:00:00')\
          .reset_index()
print(len(IBMTest))

395


In [2]:
import time
import socket
TCP_IP = '127.0.0.1'
TCP_PORT = 5000


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((TCP_IP, TCP_PORT))
print("Awaiting Connection")

s.listen(1)
print("Server started at port 5000.")
while True:
    conn, addr = s.accept()
    print("Connected to: ", addr)
    for i in range(0,len(IBMTest)):
        IBMTest1 = IBMTest[i:i+1]
        Stocks_Data =pd.merge_asof(IBMTest1, SentimentTest.sort_values('DateTime'),
                      on='DateTime',
                      tolerance=pd.Timedelta('7m'))
        Stocks_Data.drop(columns=['High','Low','Close'],inplace=True)
        Stocks_Data.fillna(0,inplace=True)
        Stocks_Data['DateTime']= Stocks_Data['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
        #print(Stocks_Data.to_json())
        print(Stocks_Data.reset_index().to_json(orient='records'))
        Stocks_Data.reset_index(drop=True, inplace=True)
        conn.send((Stocks_Data.to_json(orient='records')+'\n').encode())
        time.sleep(30)
    time.sleep(10)
    conn.close()

Awaiting Connection
Server started at port 5000.
Connected to:  ('127.0.0.1', 61927)
[{"index":0,"DateTime":"2020-03-31 09:00:00","Open":111.77,"Volume":149,"Polarity":0.0}]
[{"index":0,"DateTime":"2020-03-31 09:13:00","Open":112.05,"Volume":102,"Polarity":0.007670455}]
[{"index":0,"DateTime":"2020-03-31 09:14:00","Open":112.0,"Volume":234,"Polarity":0.007670455}]
[{"index":0,"DateTime":"2020-03-31 09:15:00","Open":111.8,"Volume":100,"Polarity":-0.031111111}]
[{"index":0,"DateTime":"2020-03-31 09:30:00","Open":112.0,"Volume":127232,"Polarity":0.0}]
[{"index":0,"DateTime":"2020-03-31 09:31:00","Open":112.27,"Volume":11257,"Polarity":0.0}]
[{"index":0,"DateTime":"2020-03-31 09:32:00","Open":112.44,"Volume":16741,"Polarity":0.0}]


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [ ]:
for i in range(0,len(IBMTest)):
    IBMTest1 = IBMTest[i:i+1]
    Stocks_Data =pd.merge_asof(IBMTest1, SentimentTest.sort_values('DateTime'),
                  on='DateTime',
                  tolerance=pd.Timedelta('7m'))
    Stocks_Data.drop(columns=['High','Low','Close'],inplace=True)
    Stocks_Data.fillna(0,inplace=True)
    print(Stocks_Data.to_json())
    print(Stocks_Data.reset_index().to_json(orient='records'))

In [4]:
from datetime import datetime

IBMTest['DateTime']= IBMTest['DateTime'].dt.strftime('%Y-%m-%d')
IBMTest.reset_index(drop=True, inplace=True)
print(IBMTest.to_json(orient='records'))

[{"DateTime":"2020-03-31","Open":111.77,"High":111.77,"Low":111.77,"Close":111.77,"Volume":149,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":112.05,"High":112.05,"Low":112.05,"Close":112.05,"Volume":102,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":112.0,"High":112.0,"Low":111.8,"Close":111.8,"Volume":234,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":111.8,"High":111.8,"Low":111.8,"Close":111.8,"Volume":100,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":112.0,"High":112.84,"Low":111.91,"Close":112.42,"Volume":127232,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":112.27,"High":112.43,"Low":111.81,"Close":112.43,"Volume":11257,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":112.44,"High":112.8,"Low":112.16,"Close":112.3,"Volume":16741,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":112.165,"High":112.27,"Low":111.69,"Close":112.27,"Volume":46745,"Date":"2020-03-31"},{"DateTime":"2020-03-31","Open":112.24,"High":112.575,"Low":112.24,"Close":112.2

In [ ]:
IBMTest[['Date', 'Close']]